<a href="https://colab.research.google.com/github/govind17/Information-Retrieval-Project/blob/main/LDA_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pyterrier as pt
import os

In [2]:

if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = 'C:/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/terrier_cord19'
if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer
    indexer = pt.index.IterDictIndexer(pt_index_path)
    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(),
                              fields=('abstract',),
                              meta=('docno',))
else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
    index = pt.IndexFactory.of(index_ref)


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [4]:
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker
import pandas as pd
from keyPhrasification import key_phrasification

monoT5 = MonoT5ReRanker(text_field='abstract')
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

def search_query(query):
  index_ref2 = pt.IndexRef.of(pt_index_path + "/data.properties")
  index2 = pt.IndexFactory.of(index_ref2)
  # print(query)
  if not pt.started():
      pt.init()
  br = pt.BatchRetrieve(index2) % 10
  pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
  search_result = pipeline.search(query)
  # print(search_result)
  cord19_docs = pd.read_csv('/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/cord19_sum.csv')
  filtered_docs = pd.merge(cord19_docs, search_result, on = "docno", how = "inner")
  searchResultswithkeys = key_phrasification(filtered_docs)
  print(searchResultswithkeys)
  return filtered_docs

search_query('covid')

C:\Users\Pritha\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
monoT5: 100%|██████████| 3/3 [00:12<00:00,  4.19s/batches]


Index(['docno', 'title', 'abstract_x', 'summary', 'key_phrases', 'qid',
       'docid', 'query', 'abstract_y', 'score', 'rank'],
      dtype='object')
Index(['docno', 'title', 'abstract', 'summary', 'KeyList'], dtype='object')


















































      docno                                              title  \
0  awitk3se  COVID-19 (Novel Coronavirus 2019) - recent tre...   
1  s4dqx9en  Plasma Metabolomic and Lipidomic Alterations A...   
2  u56ydlve  The impact of believing you have had COVID-19 ...   
3  3kmxzbm7  COVID-19 (Novel Coronavirus 2019) - recent trends   
4  l2rmmjqb  Validation of the British Society of Thoracic ...   
5  zub7xdi9  Comparison of the computed tomography findings...   
6  rxd08ouk  Cognitive, Affective, and Behavioral Construct...   
7  448tamvr  Validation of the British Society of Thoracic ...   
8  eyelcflh  Cognitive, Affective, and Behavioral Construct...   
9  1loqavom  Stability Analysis and Numerical Simulation of... 

,docno,title,abstract_x,summary,key_phrases,qid,docid,query,abstract_y,score,rank
0,awitk3se,COVID-19 (Novel Coronavirus 2019) - recent tre...,The World Health Organization (WHO) has issued...,COVID-19 is similar to Severe Acute Respirator...,NaN,1,32226,covid,The World Health Organization (WHO) has issued...,-0.161107,3
1,s4dqx9en,Plasma Metabolomic and Lipidomic Alterations A...,The pandemic of the coronavirus disease 2019 (...,COVID-19 is marked by its rapid progression fr...,NaN,1,68320,covid,The pandemic of the coronavirus disease 2019 (...,-0.267529,5
2,u56ydlve,The impact of believing you have had COVID-19 ...,Objectives: To investigate whether people who ...,The impact of believing you have had COVID-19 ...,NaN,1,75497,covid,Objectives: To investigate whether people who ...,-0.856843,9
3,3kmxzbm7,COVID-19 (Novel Coronavirus 2019) - recent trends,The World Health Organization (WHO) has issued...,COVID-19 is similar to Severe Acute Respirator...,NaN,1,93863,covid,The World Health Organization (WHO) has issued...,-0.161107,4
4,l2rmmjqb,Validation of the British Society of Thoracic ...,AIM: To validate the British Society of Thorac...,Validation of the British Society of Thoracic ...,NaN,1,109749,covid,AIM: To validate the British Society of Thorac...,-0.333039,6
5,zub7xdi9,Comparison of the computed tomography findings...,OBJECTIVES: To compare the chest computed tomo...,Comparison of the computed tomography findings...,NaN,1,112383,covid,OBJECTIVES: To compare the chest computed tomo...,-0.085978,0
6,rxd08ouk,"Cognitive, Affective, and Behavioral Construct...",This online survey study aimed to compare the ...,"In total, 533 sexual minority and 1421 heteros...",NaN,1,126243,covid,This online survey study aimed to compare the ...,-0.112808,1
7,448tamvr,Validation of the British Society of Thoracic ...,Abstract Aim To validate the British Society o...,Validation of the British Society of Thoracic ...,NaN,1,150199,covid,Abstract Aim To validate the British Society o...,-0.480262,7
8,eyelcflh,"Cognitive, Affective, and Behavioral Construct...",This online survey study aimed to compare the ...,"In total, 533 sexual minority and 1421 heteros...",NaN,1,153035,covid,This online survey study aimed to compare the ...,-0.112808,2
9,1loqavom,Stability Analysis and Numerical Simulation of...,The Aim of this research is construct the SEIR...,Stability Analysis and Numerical Simulation of...,NaN,1,171671,covid,The Aim of this research is construct the SEIR...,-0.639952,8


In [10]:
!pip install -U Flask

  Using cached Flask-2.2.2-py3-none-any.whl (101 kB)
  Using cached Werkzeug-2.2.2-py3-none-any.whl (232 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.2
    Uninstalling Werkzeug-2.0.2:
      Successfully uninstalled Werkzeug-2.0.2
  Attempting uninstall: Flask
    Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


In [13]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib
!python -m spacy download en_core_web_sm

  Cloning https://github.com/boudinfl/pke.git to c:\users\pritha\appdata\local\temp\pip-req-build-qn3sensy
  Resolved https://github.com/boudinfl/pke.git to commit 8f1d05dcc52041c9920ba0f9d5231fe6086d12c4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git 'C:\Users\Pritha\AppData\Local\Temp\pip-req-build-qn3sensy'


     ---------------------------------------- 1.9/1.9 MB 2.0 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160774 sha256=a9bd6b412b16705aa0ca36f4fe171c58e3c977a734de073e2b57272dad626bea
  Stored in directory: C:\Users\Pritha\AppData\Local\Temp\pip-ephem-wheel-cache-c2oi0mpj\wheels\d5\46\97\85535b5b449f70b6a3c8d1138ce8587345876891e25bfe7954
Successfully built pke
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
     ---------------------------------------- 12.8/12.8 MB 4.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-12-12 22:47:58.662787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-12-12 22:47:58.663095: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
import flask
import pandas as pd
from flask import request
from flask_cors import CORS
from keyPhrasification import key_phrasification


In [ ]:

app = flask.Flask(__name__)

# app.config["DEBUG"] = True
CORS(app)


# @app.before_first_request
# def _declareStuff():
#     global corpus
#     corpus_df = pd.read_csv(
#         '/Users/GovindShukla/tensorflow-env/trec_docs.csv')
#     corpus = corpus_df['text'].values


# def bm25okapi_search(tokenized_query, corpus, n_results=1):
#     """
#     Function that takes a tokenized query and prints the first 100 words of the
#     n_results most relevant results found in the corpus, based on the BM25
#     method.
#
#     Parameters
#     ----------
#     @param tokenized_query: list, array-like
#         A valid list containing the tokenized query.
#     @param bm25: BM25 object,
#         A valid object of type BM25 (BM25Okapi or BM25Plus) from the library
#         `rank-bm25`, initialized with a valid corpus.
#     @param corpus: list, array-like
#         A valid list containing the corpus from which the BM25 object has been
#         initialized. As returned from function read_corpus().
#     @param n_results: int, default = 1
#         The number of top results to print.
#     """
#
#     # We skip checking validity of arguments for now... We assume the user
#     # knows what they're doing.
#     # Tokenize the corpus
#     tokenized_corpus = [doc.split(" ") for doc in corpus]
#     # Instantiate BM25 object from the tokenized corpus
#     print('Still tokenizing...')
#     bm25 = BM25Plus(tokenized_corpus)
#
#     print('Still ranking...')
#
#     ranked = bm25.get_top_n(tokenized_query, corpus, n=n_results)
#     print('Ranking done...')
#     return ranked


@app.route('/query', methods=['GET'])
def search():
    query = request.args.get('searchString')
    print(query)
    searchResults = search_query(query)
    # print(searchResults.head())
    searchResultswithkeys = key_phrasification(searchResults)
# searchResultswithkeys.to_json(orient='records')
    return searchResultswithkeys.to_json(orient='records')


@app.route('/feedback', methods=['POST'])
def fetchFeedback():
    # corpus_df = pd.read_csv(
    #     '/Users/GovindShukla/Desktop/Information-Retrieval-Project/RankedDocuments/trec_docs_sample.csv')
    # corpus = corpus_df['text'].values
    # list = request.args.get('feedbackList')
    # # print(request.body)
    # print(list)
    # return
    feedbackJson = request.json['updates']
    relevanceList = []
    if len(feedbackJson):
        for doc in feedbackJson:
            for relevance in doc['value']:
                relevanceList.append(relevance)
    print(relevanceList)
    return relevanceList

# if __name__ == "__main__":
#     corpus_df = pd.read_csv(
#         '/Users/GovindShukla/Desktop/Information-Retrieval-Project/RankedDocuments/trec_docs_sample.csv')
#     corpus = corpus_df['text'].values
if __name__ == "__main__":
    # app.debug = True
    app.run(host='0.0.0.0',port=5021)
#
#     from flask import Flask
# app = Flask(__name__)
#
# @app.route("/")
# def hello(): return "Hello World"
#
# if __name__ == "__main__":
#   app.debug = True
#   app.run(host='0.0.0.0',port=5005)


covid


monoT5: 100%|██████████| 3/3 [00:20<00:00,  6.94s/batches]
